In [1]:
import pandas as pd
import numpy as np
import itertools

In [3]:
df0 = pd.read_csv('https://raw.githubusercontent.com/pinkocto/noteda/main/posts/SOLAR/data/OBS_ASOS_TIM_data0.csv', encoding='cp949') # 2021-01-01 ~ 2021-12-31
df1 = pd.read_csv('https://raw.githubusercontent.com/pinkocto/noteda/main/posts/SOLAR/data/OBS_ASOS_TIM_data1.csv') # 2022-01-01 ~ 2023-12-31
df2 = pd.read_csv('https://raw.githubusercontent.com/pinkocto/noteda/main/posts/SOLAR/data/test_raw.csv', encoding='cp949') # 2023-01-01 ~ 2023-01-15

`-` df_raw

In [4]:
df_raw = pd.concat([df0, df1])
df_raw

,지점,지점명,일시,일사(MJ/m2)
0,93,북춘천,2021-01-01 08:00,0.00
1,93,북춘천,2021-01-01 09:00,0.37
2,93,북춘천,2021-01-01 10:00,0.96
3,93,북춘천,2021-01-01 11:00,1.40
4,93,북춘천,2021-01-01 12:00,1.72
...,...,...,...,...
229672,283,경주시,2022-12-31 14:00:00,1.82
229673,283,경주시,2022-12-31 15:00:00,1.52
229674,283,경주시,2022-12-31 16:00:00,0.96
229675,283,경주시,2022-12-31 17:00:00,0.35


`-` 지점칼럼 삭제 // 일시 $\to$ 날짜, 시간으로 분리

In [5]:
df_temp = df_raw.assign(날짜= list(map(lambda x: x[:10],df_raw['일시'])))\
.assign(시간 = list(map(lambda x: x[11:16], df_raw['일시'])))\
.drop(['일시','지점'], axis=1).rename({'일사(MJ/m2)':'일사'},axis=1).reset_index(drop=True)
df_temp

,지점명,일사,날짜,시간
0,북춘천,0.00,2021-01-01,08:00
1,북춘천,0.37,2021-01-01,09:00
2,북춘천,0.96,2021-01-01,10:00
3,북춘천,1.40,2021-01-01,11:00
4,북춘천,1.72,2021-01-01,12:00
...,...,...,...,...
444715,경주시,1.82,2022-12-31,14:00
444716,경주시,1.52,2022-12-31,15:00
444717,경주시,0.96,2022-12-31,16:00
444718,경주시,0.35,2022-12-31,17:00


`-` 파주, 상주, 동두천, 충주, 제천은 삭제

In [6]:
df_temp = df_temp.query("지점명 not in ['파주','상주','동두천','충주','제천']").reset_index(drop=True)
df_temp

,지점명,일사,날짜,시간
0,북춘천,0.00,2021-01-01,08:00
1,북춘천,0.37,2021-01-01,09:00
2,북춘천,0.96,2021-01-01,10:00
3,북춘천,1.40,2021-01-01,11:00
4,북춘천,1.72,2021-01-01,12:00
...,...,...,...,...
420955,경주시,1.82,2022-12-31,14:00
420956,경주시,1.52,2022-12-31,15:00
420957,경주시,0.96,2022-12-31,16:00
420958,경주시,0.35,2022-12-31,17:00


# --수정

`-` 시간이 비어있지 않도록...

In [15]:
reg = df_temp['지점명'].unique().tolist() 
day = df_temp['날짜'].unique().tolist() 
# time = list(df_temp['시간'].unique())
# time = ['0{}:00'.format(i) for i in range(0,8)] + time
time = ['0{}:00'.format(i) for i in range(0,10)] + ['{}:00'.format(i) for i in range(10,24)]

In [16]:
df_temp2 = pd.DataFrame(itertools.product(reg,day,time)).rename({0:'지점명',1:'날짜',2:'시간'},axis=1).merge(df_temp,how='left').fillna(0)
df_temp2

,지점명,날짜,시간,일사
0,북춘천,2021-01-01,00:00,0.0
1,북춘천,2021-01-01,01:00,0.0
2,북춘천,2021-01-01,02:00,0.0
3,북춘천,2021-01-01,03:00,0.0
4,북춘천,2021-01-01,04:00,0.0
...,...,...,...,...
770875,경주시,2022-12-31,19:00,0.0
770876,경주시,2022-12-31,20:00,0.0
770877,경주시,2022-12-31,21:00,0.0
770878,경주시,2022-12-31,22:00,0.0


In [17]:
df_temp2[:10]

,지점명,날짜,시간,일사
0,북춘천,2021-01-01,00:00,0.00
1,북춘천,2021-01-01,01:00,0.00
2,북춘천,2021-01-01,02:00,0.00
3,북춘천,2021-01-01,03:00,0.00
4,북춘천,2021-01-01,04:00,0.00
5,북춘천,2021-01-01,05:00,0.00
6,북춘천,2021-01-01,06:00,0.00
7,북춘천,2021-01-01,07:00,0.00
8,북춘천,2021-01-01,08:00,0.00
9,북춘천,2021-01-01,09:00,0.37


In [18]:
df_temp2[:-10]

,지점명,날짜,시간,일사
0,북춘천,2021-01-01,00:00,0.00
1,북춘천,2021-01-01,01:00,0.00
2,북춘천,2021-01-01,02:00,0.00
3,북춘천,2021-01-01,03:00,0.00
4,북춘천,2021-01-01,04:00,0.00
...,...,...,...,...
770865,경주시,2022-12-31,09:00,0.41
770866,경주시,2022-12-31,10:00,1.05
770867,경주시,2022-12-31,11:00,1.52
770868,경주시,2022-12-31,12:00,1.86


`-` 시간,날짜 $\to$ 일시

In [19]:
df_temp3=df_temp2.assign(일시 = list(map(lambda x,y: x+'-'+y,df_temp2['날짜'],df_temp2['시간'])))\
.drop(['날짜','시간'],axis=1)
df_temp3

,지점명,일사,일시
0,북춘천,0.0,2021-01-01-00:00
1,북춘천,0.0,2021-01-01-01:00
2,북춘천,0.0,2021-01-01-02:00
3,북춘천,0.0,2021-01-01-03:00
4,북춘천,0.0,2021-01-01-04:00
...,...,...,...
770875,경주시,0.0,2022-12-31-19:00
770876,경주시,0.0,2022-12-31-20:00
770877,경주시,0.0,2022-12-31-21:00
770878,경주시,0.0,2022-12-31-22:00


`-` 저장

In [20]:
df_temp3.rename({'지점명':'region','일사':'solar_radiation','일시':'date'},axis=1)

,region,solar_radiation,date
0,북춘천,0.0,2021-01-01-00:00
1,북춘천,0.0,2021-01-01-01:00
2,북춘천,0.0,2021-01-01-02:00
3,북춘천,0.0,2021-01-01-03:00
4,북춘천,0.0,2021-01-01-04:00
...,...,...,...
770875,경주시,0.0,2022-12-31-19:00
770876,경주시,0.0,2022-12-31-20:00
770877,경주시,0.0,2022-12-31-21:00
770878,경주시,0.0,2022-12-31-22:00


In [21]:
df = df_temp3.rename({'지점명':'region','일사':'solar_radiation','일시':'date'},axis=1)
df.to_csv("solar_radiation2.csv",index=False)
!git add .
!git commit -m .
!git push 

[main 2af59d2] .
 13 files changed, 780980 insertions(+), 2928 deletions(-)
 create mode 100644 "posts/Study/.ipynb_checkpoints/2023-04-03-\354\235\274\354\202\254\353\237\211-checkpoint.ipynb"
 create mode 100644 posts/Study/.ipynb_checkpoints/2023-04-10-arima-ex-checkpoint.ipynb
 create mode 100644 posts/Study/.ipynb_checkpoints/2023-04-10-arima-ex-day-checkpoint.ipynb
 create mode 100644 posts/Study/.ipynb_checkpoints/2023-04-11-arima-hour-data-fill-checkpoint.ipynb
 create mode 100644 "posts/Study/2023-04-03-\354\235\274\354\202\254\353\237\211.ipynb"
 create mode 100644 posts/Study/2023-04-10-arima-ex-day.ipynb
 create mode 100644 posts/Study/2023-04-10-arima-ex.ipynb
 create mode 100644 posts/Study/2023-04-11-arima-hour-data-fill.ipynb
 create mode 100644 posts/Study/solar_radiation2.csv
Enumerating objects: 17, done.
Counting objects: 100% (17/17), done.
Delta compression using up to 16 threads
Compressing objects: 100% (13/13), done.
Writing objects: 100% (13/13), 3.21 MiB | 1.

`-` 불러오기

In [23]:
df = pd.read_csv("https://raw.githubusercontent.com/pinkocto/noteda/main/posts/SOLAR/data/solar_radiation2.csv")
df

HTTPError: HTTP Error 404: Not Found

`-` 다운로드

In [45]:
# !wget https://raw.githubusercontent.com/miruetoto/yechan3/main/posts/3_Researches/SOLAR/solar_radiation.csv